# Requirements

In [73]:
import os
import sys


def add_sys_path(p):
    p = os.path.abspath(p)
    print(p)
    if p not in sys.path:
        sys.path.append(p)

In [74]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [75]:
from importlib import reload

In [76]:
import condbert
reload(condbert)
from condbert import CondBertRewriter

In [77]:
import torch
from transformers import BertTokenizer, BertForMaskedLM
import numpy as np
import pickle
from tqdm.auto import tqdm, trange

In [78]:
device = torch.device('cuda:0')

### Load the model

In [79]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import RobertaForMaskedLM, RobertaTokenizer
model_name = "pysentimiento/robertuito-base-uncased" #Robertuito
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = RobertaForMaskedLM.from_pretrained(model_name)

/media/gabriel/Datos Linux/loncos/Text-Detoxification-in-Spanish/env/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [80]:
model.to(device);

#### Load vocabularies for spans detection

In [81]:
vocab_root = 'vocab/'

In [82]:
with open(vocab_root + "negative-words.txt", "r") as f:
    s = f.readlines()
negative_words = list(map(lambda x: x[:-1], s))
with open(vocab_root + "toxic_words.txt", "r") as f:
    ss = f.readlines()
negative_words += list(map(lambda x: x[:-1], ss))

with open(vocab_root + "positive-words.txt", "r") as f:
    s = f.readlines()
positive_words = list(map(lambda x: x[:-1], s))

In [83]:
import pickle
with open(vocab_root + 'word2coef.pkl', 'rb') as f:
    word2coef = pickle.load(f)

In [84]:
token_toxicities = []
with open(vocab_root + 'token_toxicities.txt', 'r') as f:
    for line in f.readlines():
        token_toxicities.append(float(line))
token_toxicities = np.array(token_toxicities)
token_toxicities = np.maximum(0, np.log(1/(1/token_toxicities-1)))   # log odds ratio

# discourage meaningless tokens
for tok in ['.', ',', '-']:
    token_toxicities[tokenizer.encode(tok)][1] = 3

for tok in ['you']:
    token_toxicities[tokenizer.encode(tok)][1] = 0

### Applying the model

In [85]:
reload(condbert)
from condbert import CondBertRewriter

editor = CondBertRewriter(
    model=model,
    tokenizer=tokenizer,
    device=device,
    neg_words=negative_words,
    pos_words=positive_words,
    word2coef=word2coef,
    token_toxicities=token_toxicities,
)

In [86]:
print(editor.translate("! Jacintooo...!, !!!te voy a sacar la mierda...!!!", prnt=False))

! jacintooo...!, !!!te voy a sacar la basura...!!!


### Multiunit

In [87]:
from multiword import masked_token_predictor_bert
reload(masked_token_predictor_bert)
from multiword.masked_token_predictor_bert import MaskedTokenPredictorBert

In [88]:
def adjust_logits(logits, label=0):
    return logits - token_toxicities * 100 * (1 - 2 * label)

predictor = MaskedTokenPredictorBert(model, tokenizer, max_len=250, device=device, label=0, contrast_penalty=0.0, logits_postprocessor=adjust_logits)

editor = CondBertRewriter(
    model=model,
    tokenizer=tokenizer,
    device=device,
    neg_words=negative_words,
    pos_words=positive_words,
    word2coef=word2coef,
    token_toxicities=token_toxicities,
    predictor=predictor,
)

In [89]:
%%time
print(editor.replacement_loop('! Jacintooo...!, !!!te voy a sacar la mierda ...!!!', verbose=False, n_units=1))

! jacintooo...!, !!!te voy a sacar la ...!!!
CPU times: user 39.3 ms, sys: 1.97 ms, total: 41.2 ms
Wall time: 40.3 ms


In [90]:
%%time
print(editor.replacement_loop('! Jacintooo...!, !!!te voy a sacar la mierda ...!!!', verbose=False, n_units=3))

! jacintooo...!, !!!te voy a sacar la ...!!!
CPU times: user 77.7 ms, sys: 968 μs, total: 78.6 ms
Wall time: 77.7 ms


In [91]:
%%time
print(editor.replacement_loop('! Jacintooo...!, !!!te voy a sacar la mierda...!!!', verbose=False, n_units=10))

! jacintooo...!, !!!te voy a sacar la...!!!
CPU times: user 85.3 ms, sys: 1 ms, total: 86.3 ms
Wall time: 85.7 ms


In [92]:
import choosers
reload(choosers)
from choosers import EmbeddingSimilarityChooser

In [93]:
chooser = EmbeddingSimilarityChooser(sim_coef=10, tokenizer=tokenizer)

In [94]:
print(editor.replacement_loop('! Jacintooo...!, !!!te voy a sacar la mierda...!!!', verbose=True, n_units=10, chooser=chooser))

2024-11-05 20:49:34,136 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2024-11-05 20:49:34,137 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2024-11-05 20:49:34,137 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2024-11-05 20:49:34,138 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2024-11-05 20:49:34,138 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2024-11-05 20:49:34,139 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2024-11-05 20:49:34,139 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2024-11-05 20:49:34,140 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2024-11-05 20:49:34,140 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2024-11-05 20:49:34,141 Warning: An empty Sentence was 

# Reproduction

In [96]:
import preprocessor as p

In [97]:
import pandas as pd
#dataTest = pd.read_csv('../../data/test/test.txt', sep='\t', header=None, names=['text'])
dataTest = pd.read_parquet('../../data/test/test2.parquet')
dataTest

,text
0,Quien me mando a hacerme este aro de mierda 😭
1,Tengo algo mejor que una pata de conejo.
2,te he respondido 2 veces como un subnormal xDD...
3,Me baje la discografia de Jeffree Star porque ...
4,Hay que ser muy cara raja de pedir paz si no h...
...,...
595,igual alta paja dsps se me parte una y m corto...
596,Frío de mierda me sacas las ganas de salir
597,"Yo estoy mil horas en pensar si tengo q sumar,..."
598,Si le votan a este energúmeno de verdad no ama...


In [101]:
editor.translate('quien me mando a hacerme este aro de mierda', prnt=False)

'quien me mando a hacerme este aro de mierda,'

In [98]:
test = []
for twt in dataTest['text']:
    test.append(editor.translate(p.clean(twt), prnt=False))
test

['quien me mando a hacerme este aro de mierda,',
 'Tengo algo mejor que una pata de conejo.',
 'te he respondido veces como un subnormal xDDDDDDDD',
 'Me baje la discografia de Jeffree Star porque me representa: Una mierda, pero fabulosa.',
 'Hay que ser muy cara raja de pedir paz si no hay justicia',
 'estamos de acuerdo que el terer le rompe el orto al mate?',
 'Y acaba de cagar... DONDE NO TIENE QUE HACERLO!!!!!!!!!!',
 'mis ganas de seguir se estn yendo a la própísima mierda.',
 'despus de estos das te llamarn el leador: palo q ves palo que tumbas',
 'a juzgar de las msicas new age que clavaron los vecinos, los hospis", estn hablando de ayahuasca.',
 'Che cuando sea presidente , se van a querer cortar un huevo jajaja!',
 'por todos nuestros compaeros que se han suicidado con esta carrera de mierda.',
 'A mi algunas veces se menstrujan los huevos con los vaqueros ajustados.',
 'storm es una hijapu de cuidado.',
 'se ve como una mierda, la foto perooo',
 'estoy tan hasta la polla que

In [45]:
test2 = []
for twt in dataTest['text']:
    test2.append(editor.replacement_loop(twt, verbose=False, chooser=chooser))
test2

2024-11-05 20:41:57,510 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2024-11-05 20:41:57,511 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2024-11-05 20:41:57,512 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2024-11-05 20:41:57,513 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2024-11-05 20:41:57,515 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2024-11-05 20:42:00,360 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2024-11-05 20:42:00,360 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2024-11-05 20:42:00,361 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2024-11-05 20:42:00,361 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2024-11-05 20:42:00,362 Warning: An empty Sentence was 

['quien me mando a hacerme este aro de mierda, ',
 'Tengo algo mejor que una pata de conejo.',
 'te he respondido 2 veces como un subnormal xDDDDDDDD',
 'Me baje la discografia de Jeffree Star porque me representa: Una mierda, pero fabulosa.',
 'Hay que ser muy cara raja de pedir paz si no hay justicia',
 'estamos de acuerdo que el tereré le rompe el orto al mate?',
 'Y acaba de cagar... DONDE NO TIENE QUE HACERLO!!!!!!!!!!',
 'mis ganas de seguir se están yendo a la re zarpísima mierda.',
 'Después de estos días te llamarán el Leñador: Palo q ves palo que tumbas',
 'a juzgar de las músicas new age que clamaran "por los vecinos, los hospis", están todas las canciones de ayahuasca.',
 'Che cuando sea presidente , se van a querer cortar un huevo jajaja!',
 'por todos nuestros compañeros que se han suicidado con esta carrera de mierda.  url ',
 'A mi algunas veces se menstrujan los huevos con los vaqueros ajustados.',
 'storm es una hija con muchas necesidades de cuidado.',
 'se ve como u

In [30]:
with open('../../data/test/input1.txt', 'w') as input1, open('../../data/test/input2.txt', 'w') as input2:
    for twt in test:
        input1.writelines(f'{twt}\n')
    for twt in test2:
        input2.writelines(f'{twt}\n')

In [1]:
%cd ../..

/media/gabriel/Datos Linux/loncos/Text-Detoxification-in-Spanish


/media/gabriel/Datos Linux/loncos/Text-Detoxification-in-Spanish/env/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [6]:

!python metric/metric.py --inputs data/test/test.txt --preds data/test/input1.txt

Calculating style of predictions
/media/gabriel/Datos Linux/loncos/Text-Detoxification-in-Spanish/env/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/media/gabriel/Datos Linux/loncos/Text-Detoxification-in-Spanish/env/lib/python3.10/site-packages/transformers/modeling_utils.py:399: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpic

In [7]:
!python metric/metric.py --inputs data/test/test.txt --preds data/test/input2.txt

Calculating style of predictions
/media/gabriel/Datos Linux/loncos/Text-Detoxification-in-Spanish/env/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/media/gabriel/Datos Linux/loncos/Text-Detoxification-in-Spanish/env/lib/python3.10/site-packages/transformers/modeling_utils.py:399: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpic

In [60]:
print(editor.translate("Con ese cuerpo es toda una marraqueta", prnt=False))
print(editor.replacement_loop("Con ese cuerpo es toda una marraqueta", verbose=True, n_tokens=(1, 2), n_top=10))   

con ese cuerpo es toda una marraqueta
['▁mar'] -> ['▁co', 'tor']
con ese cuerpo es toda una cotorraqueta


In [61]:
print(editor.translate("Esa chica ya se tropezó dos veces desde que entró, es una petaca", prnt=False))
print(editor.replacement_loop("Esa chica ya se tropezó dos veces desde que entró, es una petaca", verbose=False, n_tokens=(1, 2), n_top=10))   

esa chica ya se tropezó dos veces desde que entró, es una petaca
esa chica ya se tropezó dos veces desde que entró, es una peeeorca


In [62]:
print(editor.translate("El huañuli ese qué va a poder cargar tantos sillares.", prnt=False))
print(editor.replacement_loop("El huañuli ese qué va a poder cargar tantos sillares.", verbose=False, n_tokens=(1, 2), n_top=10))  

el huichuli ese qué va a poder cargar tantos sillares.
el huévuli ese qué va a poder cargar tantos sillares.
